In [2]:
import os
import pickle

from multiprocessing import Pool
import numpy as np
import mne

drop_channels = ['PHOTIC-REF', 'IBI', 'BURSTS', 'SUPPR', 'EEG ROC-REF', 'EEG LOC-REF', 'EEG EKG1-REF', 'EMG-REF', 'EEG C3P-REF', 'EEG C4P-REF', 'EEG SP1-REF', 'EEG SP2-REF', \
                 'EEG LUC-REF', 'EEG RLC-REF', 'EEG RESP1-REF', 'EEG RESP2-REF', 'EEG EKG-REF', 'RESP ABDOMEN-REF', 'ECG EKG-REF', 'PULSE RATE', 'EEG PG2-REF', 'EEG PG1-REF']
drop_channels.extend([f'EEG {i}-REF' for i in range(20, 129)])
chOrder_standard = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', \
                    'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG T1-REF', 'EEG T2-REF']


renamed_channels_dict = {
    'Fp1': 'EEG Fp1-REF', 'AF7': 'EEG AF7-REF', 'AF3': 'EEG AF3-REF', 'F1': 'EEG F1-REF',
    'F3': 'EEG F3-REF', 'F5': 'EEG F5-REF', 'F7': 'EEG F7-REF', 'FT7': 'EEG FT7-REF',
    'FC5': 'EEG FC5-REF', 'FC3': 'EEG FC3-REF', 'FC1': 'EEG FC1-REF', 'C1': 'EEG C1-REF',
    'C3': 'EEG C3-REF', 'C5': 'EEG C5-REF', 'T7': 'EEG T7-REF', 'TP7': 'EEG TP7-REF',
    'CP5': 'EEG CP5-REF', 'CP3': 'EEG CP3-REF', 'CP1': 'EEG CP1-REF', 'P1': 'EEG P1-REF',
    'P3': 'EEG P3-REF', 'P5': 'EEG P5-REF', 'P7': 'EEG P7-REF', 'P9': 'EEG P9-REF',
    'PO7': 'EEG PO7-REF', 'PO3': 'EEG PO3-REF', 'O1': 'EEG O1-REF', 'Iz': 'EEG Iz-REF',
    'Oz': 'EEG Oz-REF', 'POz': 'EEG POz-REF', 'Pz': 'EEG Pz-REF', 'CPz': 'EEG CPz-REF',
    'Fpz': 'EEG Fpz-REF', 'Fp2': 'EEG Fp2-REF', 'AF8': 'EEG AF8-REF', 'AF4': 'EEG AF4-REF',
    'AFz': 'EEG AFz-REF', 'Fz': 'EEG Fz-REF', 'F2': 'EEG F2-REF', 'F4': 'EEG F4-REF',
    'F6': 'EEG F6-REF', 'F8': 'EEG F8-REF', 'FT8': 'EEG FT8-REF', 'FC6': 'EEG FC6-REF',
    'FC4': 'EEG FC4-REF', 'FC2': 'EEG FC2-REF', 'FCz': 'EEG FCz-REF', 'Cz': 'EEG Cz-REF',
    'C2': 'EEG C2-REF', 'C4': 'EEG C4-REF', 'C6': 'EEG C6-REF', 'T8': 'EEG T8-REF',
    'TP8': 'EEG TP8-REF', 'CP6': 'EEG CP6-REF', 'CP4': 'EEG CP4-REF', 'CP2': 'EEG CP2-REF',
    'P2': 'EEG P2-REF', 'P4': 'EEG P4-REF', 'P6': 'EEG P6-REF', 'P8': 'EEG P8-REF',
    'P10': 'EEG P10-REF', 'PO8': 'EEG PO8-REF', 'PO4': 'EEG PO4-REF', 'O2': 'EEG O2-REF'
}



In [41]:
import pickle
import pandas as pd
FG_overview = pickle.load(open("/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/FG_overview_df_v2.pkl", "rb"))
FG_overview

,Subject_id,Exp_id,Friend_status,EEG_device,Force_device,Force_port,Age,Gender,Class_friends,Class_close_friends,Friends,Close_friends,Triad_id,Participant,tFriends,tClose_friends,All_friends,tClass,rClass
0,1049,301A,Yes,1,1,1,22.239562,F,3,6,9,14,301,P1,12,20,32,9,0.666667
1,1029,301B,No,2,3,2,24.331280,M,3,1,8,5,301,P2,11,6,17,4,0.250000
2,1028,301C,Yes,3,4,3,21.670089,M,8,4,9,4,301,P3,17,8,25,12,0.333333
3,1064,302A,Yes,1,1,1,22.009582,M,14,8,11,5,302,P1,25,13,38,22,0.363636
4,1024,302B,Yes,2,3,2,21.355236,M,5,7,6,10,302,P2,11,17,28,12,0.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,6069,330B,No,2,3,2,19.373032,F,14,3,17,17,330,P2,31,20,51,17,0.176471
88,6048,330C,Yes,3,4,3,19.841205,M,13,11,25,26,330,P3,38,37,75,24,0.458333
89,6015,331A,Yes,1,1,1,20.052019,M,9,4,14,13,331,P1,23,17,40,13,0.307692
90,6183,331B,Yes,2,3,2,18.863792,M,22,6,25,12,331,P2,47,18,65,28,0.214286


In [16]:
Force_df = pickle.load(open("/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/Force_df_v2.pkl", "rb"))
Force_df
unique_trial_ids = Force_df['Trial'].nunique()
print(f"Number of unique trial_id's in Force_df: {unique_trial_ids}")

Number of unique trial_id's in Force_df: 300


In [17]:
Force_df["Trial"]

0       10
1       10
2       10
3       10
4       10
      ... 
500    300
501    300
502    300
503    300
504    300
Name: Trial, Length: 4478340, dtype: int64

In [24]:
Behavior = pickle.load(open("/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/Beh_feat_df_v2.pkl", "rb"))
unique_trial_ids_behavior = Behavior['Trial'].nunique()
print(f"Number of unique trial_id's in Behavior: {unique_trial_ids_behavior}")

Number of unique trial_id's in Behavior: 300


In [39]:
Behavior[Behavior["Triad_id"] == 301][0:20]

,Triad_id,Block,Trial,Condition_id,Condition,Force_level,TargetForce,Participant,Feature,Value,Subject_id,Friend_status,Age,Gender,Class_friends,Class_close_friends,Friends,Close_friends
0,301,1,1,7,T13P,1,16,P1,Latency,2.560000,1049,Yes,22.239562,F,3,6,9,14
1,301,1,1,7,T13P,1,16,P1,Stability,1.320000,1049,Yes,22.239562,F,3,6,9,14
2,301,1,1,7,T13P,1,16,P1,Success,1.000000,1049,Yes,22.239562,F,3,6,9,14
3,301,1,1,7,T13P,1,16,P1,RelativeForce,0.721505,1049,Yes,22.239562,F,3,6,9,14
4,301,1,1,7,T13P,1,16,P1,cVariability,0.045401,1049,Yes,22.239562,F,3,6,9,14
5,301,1,2,7,T13P,4,28,P1,Latency,NaN,1049,Yes,22.239562,F,3,6,9,14
6,301,1,2,7,T13P,4,28,P1,Stability,0.000000,1049,Yes,22.239562,F,3,6,9,14
7,301,1,2,7,T13P,4,28,P1,Success,0.000000,1049,Yes,22.239562,F,3,6,9,14
8,301,1,2,7,T13P,4,28,P1,RelativeForce,NaN,1049,Yes,22.239562,F,3,6,9,14
9,301,1,2,7,T13P,4,28,P1,cVariability,0.034944,1049,Yes,22.239562,F,3,6,9,14


In [ ]:
print(Behavior["Condition"].nunique(), )

10

In [14]:
Behavior.columns

Index(['Triad_id', 'Block', 'Trial', 'Condition_id', 'Condition',
       'Force_level', 'TargetForce', 'Participant', 'Feature', 'Value',
       'Subject_id', 'Friend_status', 'Age', 'Gender', 'Class_friends',
       'Class_close_friends', 'Friends', 'Close_friends'],
      dtype='object')